In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  MinMaxScaler
import keras
from keras import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import metrics
import os

In [ ]:
os.chdir("C:\\Users\\Korisnik\\Downloads")

<font color='crimson'>

# 1. Preparing the data

### Loading and filling in missing values

In [ ]:
data = pd.read_csv('Dataset-sa-generatorom222.csv')

data=data.drop(1094,axis=0)                       #dropping the extra row which is empty
data=data.drop('Maximum speed of wind', axis=1)   #dropping the column with no entries

for i in range(1,data.shape[1]):
    data.iloc[:,i] = pd.to_numeric(data.iloc[:,i], errors='coerce')    #transforming data from "object" to "float"
    
data.iloc[:,:-1] = data.iloc[:,:-1].fillna(method='ffill')  #filling in missing values from attribute columns 

data['Output']=data.iloc[:,-1].interpolate() #interpolating missing output data

data['Day'] = pd.to_datetime(data['Day'],dayfirst=True)
data['Year'] = data.Day.map(lambda x: x.year)
data['Month'] = data.Day.map(lambda x: x.month)
data['DayOfMonth'] = data.Day.map(lambda x: x.day) #separating date instances

data = data.drop('Day', axis = 'columns')  

cols=data.columns.tolist()
cols = cols[-3:] + cols[:-3]
data=data[cols]

In [ ]:
data.head(3)

### Chencking if there are correlated columns

In [ ]:
sns.pairplot(data.iloc[:,3:])

In [ ]:
data= data.drop(['Maximum temperature','Minimum temperature', 'Maximum sustained wind speed'],axis=1)

### Since it is recommended that neural network input be a matrix with columns which are not highly correlated, we dropped the three columns which had high correlation with other columns.

### Sorting the data into seperate sets for crossvalidation:

In [ ]:
x_train_2013   =  data[data['Year']!=2013]
y_train_2013   =  x_train_2013['Output']
x_train_2013   =  x_train_2013.iloc[:,3:-1]

x_train_2014   =  data[data['Year']!=2014]
y_train_2014   =  x_train_2014['Output']
x_train_2014   =  x_train_2014.iloc[:,3:-1]

x_train_2015   =  data[data['Year']!=2015]
y_train_2015   =  x_train_2015['Output']
x_train_2015   =  x_train_2015.iloc[:,3:-1]

In [ ]:
x_test_2013    =  data[data['Year']==2013]
y_test_2013    =  x_test_2013['Output']
x_test_2013    =  x_test_2013.iloc[:,3:-1]

x_test_2014    =  data[data['Year']==2014]
y_test_2014    =  x_test_2014['Output']
x_test_2014    =  x_test_2014.iloc[:,3:-1]

x_test_2015    =  data[data['Year']==2015]
y_test_2015    =  x_test_2015['Output']
x_test_2015    =  x_test_2015.iloc[:,3:-1]

###  Scaling input and output data:

In [ ]:
sc= MinMaxScaler()

x_train_2013   =  sc.fit_transform(x_train_2013)
x_train_2014   =  sc.fit_transform(x_train_2014)
x_train_2015   =  sc.fit_transform(x_train_2015)

x_test_2013    =  sc.fit_transform(x_test_2013)
x_test_2014    =  sc.fit_transform(x_test_2014)
x_test_2015    =  sc.fit_transform(x_test_2015)

y_train_2013   =  np.array(y_train_2013).reshape(-1,1)
y_train_2013   =  sc.fit_transform(y_train_2013)
y_train_2014   =  np.array(y_train_2014).reshape(-1,1)
y_train_2014   =  sc.fit_transform(y_train_2014)
y_train_2015   =  np.array(y_train_2015).reshape(-1,1)
y_train_2015   =  sc.fit_transform(y_train_2015)

y_test_2013    =  np.array(y_test_2013).reshape(-1,1)
y_test_2013    =  sc.fit_transform(y_test_2013)
y_test_2014    =  np.array(y_test_2014).reshape(-1,1)
y_test_2014    =  sc.fit_transform(y_test_2014)
y_test_2015    =  np.array(y_test_2015).reshape(-1,1)
y_test_2015    =  sc.fit_transform(y_test_2015)

<font color = 'crimslon'>

# 2. Defining the Neural Network:

In [ ]:
def sigmoid(x):
    return(1/(1+np.exp(-x)))

def sigmoid_derivative(x):
    return(sigmoid(x)*(1-sigmoid(x)))

def add_ones(a):                          #this function ads a column of ones as the first column of the input array
    b=np.ones((a.shape[0],a.shape[1]+1))
    b[:,1:]=a
    return(b) 

In [ ]:
class NeuralNetwork():
    
    def __init__(self, x, y):
        self.input      = add_ones(x)
        np.random.seed(2)
        self.weights1   = np.random.rand(self.input.shape[1],15) 
        self.weights2   = np.random.rand(16,1) 
        self.y          = np.array(y)
        self.output     = np.zeros(self.y.shape)
        
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(add_ones(self.layer1), self.weights2))
        
    def calculate_gradients(self):
        self.d_weights2 = np.dot(add_ones(self.layer1).T, ((self.y - self.output) * sigmoid_derivative(self.output)))
        self.d_weights1 = np.dot(self.input.T,  (np.dot((self.y - self.output) * sigmoid_derivative(self.output), self.weights2[1:].T) * sigmoid_derivative(self.layer1)))
    
    def backprop(self, mu=1):
        self.calculate_gradients()
        self.weights1+=mu*self.d_weights1
        self.weights2+=mu*self.d_weights2  
        
    def backprop_corrected(self, mu, alpha=0.9):         #this function is used to speed up iteration procedure
        d_weights2_old = self.d_weights2
        d_weights1_old = self.d_weights1
        
        self.calculate_gradients()
        
        self.weights1 += alpha*(d_weights1_old) + (1-alpha)*mu*self.d_weights1
        self.weights2 += alpha*(d_weights2_old) + (1-alpha)*mu*self.d_weights2
                
        
    def train(self, mu=1 ,nr_it=10):
        for i in range(nr_it):
            self.feedforward()
            self.backprop(mu)
            
            
    def train_corrected(self, mu=1, alpha=0.9, nr_it=10):
        self.feedforward()
        self.backprop(mu)
        for i in range(nr_it-1):
            self.feedforward()
            self.backprop_corrected(mu, alpha)
            
    def predict_y(self,x):
        return(sigmoid(np.dot(add_ones(sigmoid(np.dot(add_ones(x),self.weights1))),self.weights2)))
         

### Defining a small dataset and testing our algorithm:

In [ ]:
x1=np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
y1=np.array([[0],[1],[1],[0]])

In [ ]:
NN1=NeuralNetwork(x1,y1)
NN1.train(1, nr_it=10000)

plt.figure(figsize=(12,7))
plt.scatter(range(y1.size),y1)
#plt.plot(range(trainingdata2013_output.size),trainingdata2013_output)
plt.scatter(range(NN1.output.size), NN1.output)
plt.legend(['original output', 'predicted output'])

In [ ]:
blj = pd.DataFrame(NN1.output, columns=['prediction'])
blj['y']=y1
blj

### We can see that the algorithm works!

### We now try to train three neural networks to predict data for three seperate years:

In [ ]:
NN2013 = NeuralNetwork(x_train_2013,y_train_2013)
NN2013.train(mu=0.013, nr_it=1500)
y_pred_2013 = NN2013.predict_y(x_test_2013)
e2013 = metrics.mean_absolute_error(y_pred_2013, y_test_2013)

NN2014 = NeuralNetwork(x_train_2014,y_train_2014)
NN2014.train(mu=0.013, nr_it=1500)
y_pred_2014 = NN2014.predict_y(x_test_2014)
e2014 = metrics.mean_absolute_error(y_pred_2014, y_test_2014)

NN2015 = NeuralNetwork(x_train_2015,y_train_2015)
NN2015.train(mu=0.013, nr_it=1500)
y_pred_2015 = NN2015.predict_y(x_test_2015)
e2015 = metrics.mean_absolute_error(y_pred_2015, y_test_2015)

### For reference, we use built-in functions from the Keras package:

In [ ]:
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=6, input_dim=6))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='sgd', loss='mean_squared_error',  metrics=['mae','accuracy'])
    return regressor

In [ ]:
NN2013_keras = KerasRegressor(build_fn=build_regressor, batch_size=730,epochs=1500)
NN2013_keras.fit(x_train_2013, y_train_2013)
y_pred_2013_keras = NN2013_keras.predict(x_test_2013)
e2013_keras = metrics.mean_absolute_error(y_pred_2013_keras, y_test_2013)


NN2014_keras = KerasRegressor(build_fn=build_regressor, batch_size=730,epochs=1500)
NN2014_keras.fit(x_train_2014, y_train_2014)
y_pred_2014_keras= NN2014_keras.predict(x_test_2014)
e2014_keras = metrics.mean_absolute_error(y_pred_2014_keras, y_test_2014)

NN2015_keras = KerasRegressor(build_fn=build_regressor, batch_size=729,epochs=1500)
NN2015_keras.fit(x_train_2015, y_train_2015)
y_pred_2015_keras= NN2015_keras.predict(x_test_2015)
e2015_keras = metrics.mean_absolute_error(y_pred_2015_keras, y_test_2015)

### On the following plot we can se comparisons of real output, our predictions and keras predictions:

In [ ]:
plt.figure(figsize=(12,20))

plt.subplot(3,1,1)
plt.plot(range(y_test_2013.size),y_test_2013)
plt.plot(range(y_pred_2013.size),y_pred_2013)
plt.plot(y_pred_2013_keras)
plt.legend(['real output', 'prediction our function', 'prediction built-in function'])
plt.title('Prediction for year 2013')


plt.subplot(3,1,2)
plt.plot(range(y_test_2014.size),y_test_2014)
plt.plot(range(y_pred_2014.size),y_pred_2014)
plt.plot(y_pred_2014_keras)
plt.legend(['real output', 'prediction our function', 'prediction built-in function'])
plt.title('Prediction for year 2014')

plt.subplot(3,1,3)
plt.plot(range(y_test_2015.size),y_test_2015)
plt.plot(range(y_pred_2015.size),y_pred_2015)
plt.plot(y_pred_2015_keras)
plt.legend(['real output', 'prediction of our function', 'prediction of built-in function'])
plt.title('Prediction for year 2015')
plt.show()


###  We can compare the performance of our NN against Keras NN:

In [ ]:
MAE = [e2013, e2014, e2015]
KERAS_MAE = [e2013_keras, e2014_keras, e2015_keras]

ind = np.arange(3)  # the x locations for the groups
width = 0.2

fig, ax = plt.subplots(figsize = (12,7))
rects1 = ax.bar(ind - width/2, MAE, width, label='Our NN', color = 'pink')
rects2 = ax.bar(ind + width/2, KERAS_MAE, width, label='Keras NN', color = 'magenta')

ax.set_ylabel('Mean Absolute Error')
ax.set_xlabel('Years')
ax.set_title('Error of our NN vs. error of keras NN')
ax.set_xticks(ind)
ax.set_xticklabels(('2013', '2014', '2015'))
ax.legend()
plt.show()
plt.savefig('mse', dpi = 100)

<font color='crimson'>

# 4. Crossvalidation: 

### We will use construct different models using different parameters for step size in gradient descent $\mu$ and number of iterations, and then using crossvalidation method to estimate the error of our prediction:

In [ ]:
nr_iterations = [500,1000,2000,2500]
step_sizes = [0.001, 0.005, 0.01, 0.015, 0.02, 0.1]
errors=pd.DataFrame([], columns=nr_iterations, index = step_sizes)

for i in nr_iterations:
    for j in step_sizes:
        NN2013 = NeuralNetwork(x_train_2013,y_train_2013)
        NN2013.train(mu=j, nr_it=i)
        y_pred_2013 = NN2013.predict_y(x_test_2013)
        e2013 = metrics.mean_absolute_error(y_pred_2013, y_test_2013)

        NN2014 = NeuralNetwork(x_train_2014,y_train_2014)
        NN2014.train(mu=j, nr_it=i)
        y_pred_2014 = NN2014.predict_y(x_test_2014)
        e2014 = metrics.mean_absolute_error(y_pred_2014, y_test_2014)

        NN2015 = NeuralNetwork(x_train_2015,y_train_2015)
        NN2015.train(mu=j, nr_it=i)
        y_pred_2015 = NN2015.predict_y(x_test_2015)
        e2015 = metrics.mean_absolute_error(y_pred_2015, y_test_2015)

        errors.loc[j,i] =  (e2013+e2014+e2015)/3


In [ ]:
errors